In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from rplidar import RPLidar

%matplotlib ipympl

In [2]:
cellSize = 100 # mm
maxRange = 4000 # mm

LiDAR2DPose = [-100, 200, np.pi/2]

generateRandomData = False

In [3]:
if generateRandomData:
    scanFrame = np.zeros((300, 3))

    scanFrame[:, 0] = np.random.randint(0, 50, len(scanFrame))
    scanFrame[:, 1] = np.random.uniform(0, 360, len(scanFrame))
    scanFrame[:, 2] = np.random.uniform(3000, 3500, len(scanFrame))

    qMask = scanFrame[:, 0] == 0
    rMask = scanFrame[:, 2] < 50

    mask = qMask | rMask

    scanFrame = scanFrame[~mask]

else:
    LiDAR = RPLidar("COM3")

    frameScanner = LiDAR.iter_scans()

    scanFrame = np.array(next(frameScanner))

print(len(scanFrame))
print(scanFrame[:10])

141
[[  15.         54.78125   604.5     ]
 [  15.         55.953125  611.25    ]
 [  15.         57.28125   618.25    ]
 [  15.         58.71875   626.      ]
 [  15.         60.03125   634.75    ]
 [  15.         62.640625  665.      ]
 [  15.         88.34375  1250.      ]
 [  15.         89.734375 1234.75    ]
 [  15.         92.5      1164.25    ]
 [  15.         93.8125   1139.75    ]]


In [4]:
if not generateRandomData:
    scanFrame = np.array(next(frameScanner))
    print(scanFrame[:10])

[[ 15.         1.875    602.25    ]
 [ 15.         3.25     594.75    ]
 [ 15.         4.703125 588.      ]
 [ 15.         7.328125 579.5     ]
 [ 15.         8.671875 571.5     ]
 [ 15.        10.203125 567.5     ]
 [ 15.        11.515625 563.75    ]
 [ 15.        12.828125 559.75    ]
 [ 15.        14.21875  556.25    ]
 [ 15.        15.453125 553.25    ]]


In [5]:
def convertToCartesian(scanFrame):
    xScanFrame = scanFrame[:, 2] * np.cos(scanFrame[:, 1] * np.pi / 180)
    yScanFrame = scanFrame[:, 2] * np.sin(scanFrame[:, 1] * np.pi / 180)

    return xScanFrame, yScanFrame

xScanFrame, yScanFrame = convertToCartesian(scanFrame)

In [6]:
def plotInteractive(xAxis, yAxis, color = None):
    plt.figure(figsize=(7,7))
    plt.scatter(xAxis, yAxis, c = color)
    plt.xlim(LiDAR2DPose[0] - maxRange, LiDAR2DPose[0] + maxRange)
    plt.ylim(LiDAR2DPose[1] - maxRange, LiDAR2DPose[1] + maxRange)
    plt.show()

In [7]:
plotInteractive(xScanFrame, yScanFrame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
def offsetHeading(scanFrame, LiDAR2DPose):
    scanFrame[:,1] = scanFrame[:,1] + LiDAR2DPose[2]*180/np.pi

    scanFrame[:, 1][scanFrame[:, 1]>360] -= 360

    return scanFrame

scanFrame = offsetHeading(scanFrame, LiDAR2DPose)
print(len(scanFrame[scanFrame[:,1]>360]))
print(scanFrame[:10])

0
[[ 15.        91.875    602.25    ]
 [ 15.        93.25     594.75    ]
 [ 15.        94.703125 588.      ]
 [ 15.        97.328125 579.5     ]
 [ 15.        98.671875 571.5     ]
 [ 15.       100.203125 567.5     ]
 [ 15.       101.515625 563.75    ]
 [ 15.       102.828125 559.75    ]
 [ 15.       104.21875  556.25    ]
 [ 15.       105.453125 553.25    ]]


In [9]:
def convertHeadingToRad(scanFrame):
    scanFrame[:, 1] = scanFrame[:, 1]*np.pi/180

    return scanFrame

scanFrame = convertHeadingToRad(scanFrame)
print(len(scanFrame[scanFrame[:,1]>2*np.pi]))
print(scanFrame[:10])

0
[[ 15.           1.60352125 602.25      ]
 [ 15.           1.62751953 594.75      ]
 [ 15.           1.65288134 588.        ]
 [ 15.           1.69869624 579.5       ]
 [ 15.           1.7221491  571.5       ]
 [ 15.           1.74887445 567.5       ]
 [ 15.           1.7717819  563.75      ]
 [ 15.           1.79468934 559.75      ]
 [ 15.           1.81896033 556.25      ]
 [ 15.           1.84050424 553.25      ]]


In [10]:
print(len(scanFrame))

171


In [11]:
def preProcessScanFrame(frameScanner, LiDAR2DPose):
    scanFrame = np.array(next(frameScanner))
    scanFrame = offsetHeading(scanFrame, LiDAR2DPose)
    scanFrame = convertHeadingToRad(scanFrame)

    return scanFrame

In [12]:
def generateBeliefMap(scanFrame, LiDAR2DPose):
    beliefMapUpdate = np.array([0, 0, 0, 0])

    for measure in scanFrame:
        measuredRange, direction = measure[2], measure[1]
        for rangeStep in range(0, int(measuredRange+cellSize), cellSize):
            belief = 0.3

            if rangeStep > measuredRange: 
                belief = 0.7
                rangeStep = measuredRange

            xCell = ((LiDAR2DPose[0] + rangeStep*np.cos(direction))//cellSize) * cellSize
            yCell = ((LiDAR2DPose[1] + rangeStep*np.sin(direction))//cellSize) * cellSize

            if beliefMapUpdate.size > 6:
                xMask = beliefMapUpdate[:, 0] == xCell
                yMask = beliefMapUpdate[:, 1] == yCell

                mask = xMask&yMask
                index = np.argmax(mask)
            
                if any(mask):
                    # Updating belief
                    beliefMapUpdate[index, 3] = beliefMapUpdate[index, 3] + 1
                    count = beliefMapUpdate[index, 3]
                    beliefMapUpdate[index, 2] = beliefMapUpdate[index, 2] * count / (count + 1) + \
                                                    belief / (count + 1)
                
                else:
                    beliefMapUpdate = np.vstack([beliefMapUpdate, np.array([xCell, yCell, belief, 0])])

            else:
                beliefMapUpdate = np.vstack([beliefMapUpdate, np.array([xCell, yCell, belief, 0])])

    beliefMapUpdate = beliefMapUpdate[1:]

    return beliefMapUpdate

beliefMapUpdate = generateBeliefMap(scanFrame, LiDAR2DPose)

print(len(beliefMapUpdate))
print(len(beliefMapUpdate[beliefMapUpdate[:, 3]==0]))
print(beliefMapUpdate[:10])

808
224
[[-1.00e+02  2.00e+02  3.00e-01  1.85e+02]
 [-2.00e+02  2.00e+02  3.00e-01  4.40e+01]
 [-2.00e+02  3.00e+02  3.00e-01  1.90e+01]
 [-2.00e+02  4.00e+02  3.00e-01  1.20e+01]
 [-2.00e+02  5.00e+02  3.00e-01  8.00e+00]
 [-2.00e+02  6.00e+02  3.00e-01  6.00e+00]
 [-2.00e+02  7.00e+02  6.20e-01  4.00e+00]
 [-2.00e+02  8.00e+02  7.00e-01  0.00e+00]
 [-3.00e+02  7.00e+02  7.00e-01  8.00e+00]
 [-3.00e+02  6.00e+02  3.00e-01  8.00e+00]]


In [13]:
print("Position:", LiDAR2DPose[0:2])
print("Heading:", LiDAR2DPose[2]*180/np.pi, "degrees")

Position: [-100, 200]
Heading: 90.0 degrees


In [14]:
plotInteractive(beliefMapUpdate[:, 0], beliefMapUpdate[:, 1], \
                                        color = beliefMapUpdate[:, 2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plotInteractive(xScanFrame, yScanFrame)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, numpoints=50):
        self.numpoints = numpoints
        self.stream = self.data_stream()

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=5, 
                                          init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        x, y, s, c = next(self.stream).T
        self.scat = self.ax.scatter(x, y, c=c, s=s, vmin=0, vmax=1,
                                    cmap="jet", edgecolor="k")
        self.ax.axis([-10, 10, -10, 10])
        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        xy = (np.random.random((self.numpoints, 2))-0.5)*10
        s, c = np.random.random((self.numpoints, 2)).T
        while True:
            xy += 0.03 * (np.random.random((self.numpoints, 2)) - 0.5)
            s += 0.05 * (np.random.random(self.numpoints) - 0.5)
            c += 0.02 * (np.random.random(self.numpoints) - 0.5)
            yield np.c_[xy[:,0], xy[:,1], s, c]


    def update(self, i):
        """Update the scatter plot."""
        data = next(self.stream)

        # Set x and y data...
        self.scat.set_offsets(data[:, :2])
        # Set sizes...
        self.scat.set_sizes(300 * abs(data[:, 2])**1.5 + 100)
        # Set colors..
        self.scat.set_array(data[:, 3])

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,


if __name__ == '__main__':
    #a = AnimatedScatter()
    #plt.show()
    pass


In [17]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, frameScanner):
        self.frameScanner = frameScanner

        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=5, 
                                          init_func=self.setup_plot, blit=True)
                                          

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        self.scat = self.ax.scatter(LiDAR2DPose[0], LiDAR2DPose[1], c=0, vmin=0, vmax=1)
        self.ax.axis([LiDAR2DPose[0] - maxRange, LiDAR2DPose[0] + maxRange, \
                      LiDAR2DPose[1] - maxRange, LiDAR2DPose[1] + maxRange])
        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,


    def update(self, i):
        """Update the scatter plot."""
        scanFrame = np.array(next(self.frameScanner))
        scanFrame = offsetHeading(scanFrame, LiDAR2DPose)
        scanFrame = convertHeadingToRad(scanFrame)
        beliefMapUpdate = generateBeliefMap(scanFrame, LiDAR2DPose)

        # Set x and y data...
        self.scat.set_offsets(beliefMapUpdate[:, :2])
        # Set colors..
        self.scat.set_array(beliefMapUpdate[:, 2])

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,


if __name__ == '__main__' and not generateRandomData:
    frameScanner = LiDAR.iter_scans() 

    scanFrame = np.array(next(frameScanner))

    a = AnimatedScatter(frameScanner)
    plt.show()
    pass


Too many bytes in the input buffer: 22525/3000. Cleaning buffer...


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …